#### Regression

In [1]:
import pandas as pd
import numpy as np

In [7]:
cars = pd.read_csv('data.csv')

In [8]:
cars.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [9]:
cars.columns

Index(['Make', 'Model', 'Year', 'Engine Fuel Type', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Driven_Wheels',
       'Number of Doors', 'Market Category', 'Vehicle Size', 'Vehicle Style',
       'highway MPG', 'city mpg', 'Popularity', 'MSRP'],
      dtype='object')

In [10]:
cars = cars.columns.str.lower().str.replace(' ', '_', regex=True)

In [14]:
strings = list(cars.dtypes[cars.dtypes == 'object'].index)

AttributeError: 'Index' object has no attribute 'dtypes'

In [12]:
cars.dtypes

AttributeError: 'Index' object has no attribute 'dtypes'

In [11]:
cars.dtypes

AttributeError: 'Index' object has no attribute 'dtypes'

We are interested in objects. 

In [13]:
cars.dtypes[cars.dtypes == 'object']

AttributeError: 'Index' object has no attribute 'dtypes'

In [9]:
strings = list(cars.dtypes[cars.dtypes == 'object'].index)

AttributeError: 'Index' object has no attribute 'dtypes'

In [10]:
for col in strings:
    cars[col] = cars[col].str.lower().str.replace(' ', '_', regex=True)

NameError: name 'strings' is not defined

#### Exploratory Data Analysis

In [ ]:
for col in cars.columns:
    print(col)
    print(cars[col].unique()[:5])
    print(cars[col].nunique())
    print()

Make
['bmw' 'audi' 'fiat' 'mercedes-benz' 'chrysler']
48

Model
['1_series_m' '1_series' '100' '124_spider' '190-class']
914

Year
[2011 2012 2013 1992 1993]
28

Engine Fuel Type
['premium_unleaded_(required)' 'regular_unleaded'
 'premium_unleaded_(recommended)' 'flex-fuel_(unleaded/e85)' 'diesel']
10

Engine HP
[335. 300. 230. 320. 172.]
356

Engine Cylinders
[ 6.  4.  5.  8. 12.]
9

Transmission Type
['manual' 'automatic' 'automated_manual' 'direct_drive' 'unknown']
5

Driven_Wheels
['rear_wheel_drive' 'front_wheel_drive' 'all_wheel_drive'
 'four_wheel_drive']
4

Number of Doors
[ 2.  4.  3. nan]
3

Market Category
['factory_tuner,luxury,high-performance' 'luxury,performance'
 'luxury,high-performance' 'luxury' 'performance']
71

Vehicle Size
['compact' 'midsize' 'large']
3

Vehicle Style
['coupe' 'convertible' 'sedan' 'wagon' '4dr_hatchback']
16

highway MPG
[26 28 27 25 24]
59

city mpg
[19 20 18 17 16]
69

Popularity
[3916 3105  819  617 1013]
48

MSRP
[46135 40650 36350 29450 345